## Problem Set 6: TM

csc427: Theory of Automata and Complexity. 
<br>
university of miami
<br>
spring 2021.
<br>
Burton Rosenberg.
<br>
<br>
created: 6 April 2021
<br>last update: 14 April 2021


---

### Student name: Temuulen Ganbold csc427 ps6

---

### TuringMachine class

In [1]:
import string
import sys
import os
import argparse
import re

#
# tm-sim.py
#
# author: bjr
# date: 21 mar 2020
# last update: 22 mar 2020
#    16 mar 2021, updated 
#     3 apr 2021, return conventions for accept/reject
#                 verbose_levels reimplemented
#                 character # is not allowed as a tape symbol
#                 for magical reasons, then " is also not allowed
#                 added class method help()
#                 
#
# copyright: Creative Commons. See http://www.cs.miami.edu/home/burt
#

# GRAMMAR for the TM description

# Comments (not shown in BNF) begin with a hash # and continue to the end
#    of the line
# The ident tokens are states
# The symbol tokens are tape symbolss
# The StateTransition semantics is:
#     tape_symbol_read tape_symbol_written action new_state
# The underscore _ is a tape blank:
# The : in a transition rule is the default tape symbol match when there is no
#    exactly matching transition rule; in the target section of the rule it 
#    is the value of the matchined tape symbol.

# A missing transition is considered a reject, not an error

class TuringMachine:
    
    verbose_levels = {'none':0,'verbose':1,'explain':2, 'debug':3}
    result_reasons = ['ok', 'transition missing', 'time limit']

    grammar = """
    M-> (Stanza [emptyline])*
    Stanza-> StartStanza | AcceptStanza | RejectStanza | StateStanze
    StartStanza-> "start" ":" ident
    AcceptStanza-> "accept" ":" ident ([newline] [indent] ident])*
    RejectStanza-> "reject" ":" ident ([newline] [indent] ident])*
    StateStanze-> "state" ":" ident ([newline] [indent] StateTransition)+
    StateTransition-> (symbol|special) (symbol|special) action ident
    action-> l|r|n|L|R|N
    symbol-> \w[!$-/]     # note: a tape symbol
    special-> ":"
    ident-> \w+           # note: name of a state

    """

    def __init__(self):
        self.start_state = "" # is an state identifier
        self.accept_states = set() # is a set of state identifiers
        self.reject_states = set() # is a set of state identifiers
        self.transitions = {} # is a map of (state,symbol):(state,symbol,action)
        self.current_state = "" 
        self.step_counter = 0
        self.all_actions = ["r","l","n"]
        self.tape = ['_']  # is a list of symbols
        self.position = 0
        self.verbose = 0
        self.result = 0

    def set_start_state(self,state):
        self.start_state = state

    def set_tape(self,tape_string):
        self.tape =  ['_' if symbol==':' or symbol==' ' else 
                          symbol for symbol in tape_string]

    def add_accept_state(self,state):
        self.accept_states.add(state)

    def add_reject_state(self,state):
        self.reject_states.add(state)
    
    def get_current_state(self):
        return self.curent_state

    def add_transition(self,state_from,read_symbol,
                       write_symbol,action,state_to):

        if action.lower() not in self.all_actions:
            # return something instead, nobody likes a chatty program
            return "WARNING: unrecognized action."
        x = (state_from, read_symbol)
        if x in self.transitions:
            # return something instead, nobody likes a chatty program
            return "WARNING: multiple outgoing states not allowed for DFA's."
        self.transitions[x] = (state_to,write_symbol,action)
        return None

    def restart(self,tape_string):
        self.current_state = self.start_state
        self.position = 0
        if len(tape_string)==0 :
            tape_string = '_'
        self.set_tape(tape_string)
        self.step_counter = 1

    def step_transition(self):
        c_s = self.current_state
        x = (c_s,self.tape[self.position])
        
        if x in self.transitions:
            (new_state, symbol, action ) = self.transitions[x]
        elif (c_s,':') in self.transitions:
            # wildcard code
            (new_state, symbol, action ) = self.transitions[(c_s,':')]
        else:
            # here we implement a rejection of convenience, if there is
            # no transition, tansition target is (:, n, A_REJECT_STATE)
            self.reason = 1
            return False
        
        # wildcard code
        if symbol==':':
            symbol = self.tape[self.position]

        self.current_state = new_state
        self.tape[self.position] = symbol

        shout = False
        if action.lower() != action:
            shout = True
            action = action.lower()
        
        if action == 'l' and self.position>0:
            self.position -= 1
        if action == 'r':
            self.position += 1
            if self.position==len(self.tape):
                self.tape[self.position:] = '_'
        if action == 'n':
            pass
   
        if shout or self.verbose == TuringMachine.verbose_levels['explain']:
            self.print_tape()
        if self.verbose == TuringMachine.verbose_levels['debug']:
            print("\t", self.step_counter, "\t", new_state, symbol, action)
            
        self.step_counter += 1
        return True

    def compute_tm(self,tape_string,step_limit=0,verbose='none'):
        self.verbose = TuringMachine.verbose_levels[verbose]
        self.result = 0
        self.restart(tape_string)
        if self.verbose == TuringMachine.verbose_levels[verbose]:
            self.print_tape()
        step = 0
            
        stop_states = self.accept_states.union(self.reject_states)
        while self.current_state not in stop_states:
            res = self.step_transition()
            if not res:
                # missing transition is considered a reject
                return False
            step += 1
            if step > step_limit:
                self.result = 2 
                return None
            
            if self.verbose == TuringMachine.verbose_levels['debug']:
                print(step, self.current_state, self.position, self.tape )

        if self.current_state in self.accept_states:
            return True
        return False

    def print_tape(self):
        t, p = self.tape, self.position
        s = ''.join(t[:p] + ['['] + [t[p]] + [']'] + t[p+1:])
        print(f'{self.current_state}:\t{s}')
    
    def print_tm(self):
        print("\nstart state:\n\t",self.start_state)
        print("accept states:\n\t",self.accept_states)
        print("reject states:\n\t",self.reject_states)
        print("transitions:")
        for t in self.transitions:
            print("\t",t,"->",self.transitions[t])
    
    @classmethod
    def help(cls):
        print('The verbose levels are:')
        for level in cls.verbose_levels:
            print(f'\t{cls.verbose_levels[level]}: {level}')
        print()
        print('The grammar for the Turing Machine description is:')
        print(cls.grammar)
        
        
### end class TuringMachine


class MachineParser:

    @staticmethod
    def turing(tm_obj, fa_string):
        """
        Code to parse a Turing Machine description into the Turing Machine object.
        """
        
        fa_array = fa_string.splitlines()
        line_no = 0 
        current_state = ""
        in_state_read = False
        in_accept_read = False
        in_reject_read = False

        for line in fa_array:
            while True:

                # comment lines are fully ignored
                if re.search('^\s*#',line):
                    break

                if re.search('^\s+',line):

                    if in_state_read:
                        m = re.search('\s+(\w|[!$-/:])\s+(\w|[!$-/:])\s+(\w)\s+(\w+)',line)
                        if m:
                            res = tm_obj.add_transition(current_state,
                                    m.group(1),m.group(2),m.group(3),m.group(4))
                            if res: 
                                print(res, f'line number {line_no}')
                                return False
                            break

                    if in_accept_read:
                        m = re.search('\s+(\w+)',line)
                        if m:
                            tm_obj.add_accept_state(m.group(1))
                            break

                    if in_reject_read:
                        m = re.search('\s+(\w+)',line)
                        if m:
                            tm_obj.add_reject_state(m.group(1))
                            break

                in_state_read = False
                in_accept_read = False
                in_reject_read = False

                # blank lines do end multiline input
                if re.search('^\s*$',line):
                    break ;

                m = re.search('^start:\s*(\w+)',line)
                if m:
                    tm_obj.set_start_state(m.group(1))
                    break

                m = re.search('^accept:\s*(\w+)',line)
                if m:
                    tm_obj.add_accept_state(m.group(1))
                    in_accept_read = True
                    break

                m = re.search('^reject:\s*(\w+)',line)
                if m:
                    tm_obj.add_reject_state(m.group(1))
                    in_reject_read = True
                    break

                m = re.search('^state:\s*(\w+)',line)
                if m:
                    in_state_read = True
                    current_state = m.group(1)
                    break

                print(line_no,"warning: unparsable line, dropping: ", line)
                return False
                break

            line_no += 1
        return True

### end class MachineParser



In [2]:

def create_and_test_turing_machine(tm_description, test_cases,verbose='none'):
    tm = TuringMachine()
    MachineParser.turing(tm,tm_description)
 
    print("\n*** TEST RUNS ***")

    for s in test_cases:
        # assume complexity is some quadratic
        res = tm.compute_tm(s,step_limit=10*(len(s)+5)**2,verbose=verbose)
        if res==True:
            print(f'ACCEPT input {s}\n')
        elif res==False:
            print(f'REJECT input {s}\n')
        else:
            print(f'ERROR on input {s}: {TuringMachine[tm.result]}')
            
    print("\n\n*** RUN COMPLETE ***\n\n")

# TuringMachine.help()

## Exercise A

In [3]:
# Turing Machine M3, Sipser 3ird ed page 174, Sipser 2nd ed page 146

tm_M3 = """# The language of multiplication
# a^i b^j c^k, i,j,k >=1, and k = i*j

# a student assignment 

start: q1
accept: A
reject: R

state: q1
    a _ r q2
    
state: q2
    a : r q2
    b : r q3
    
state: q3
    b : r q3
    c : r q4

state: q4
    c : r q4
    _ : l q5

state: q5
    c : l q5
    b : l q5
    a : l q5
    _ : r q6
    
state: q6
    a : r q6
    b B r q7
    x : l q9
    
state: q7
    b : r q7
    x : r q7
    c x l q8
    
state: q8
    x : l q8
    b : l q8
    B : r q6
    
state: q9
    B b l q9
    a : l q9
    _ : r q10
    
state: q10
    a _ r q6
    b : r q11

state: q11
    b : r q11
    x : r q11
    _ : r A
"""

tm_M3_test = [
    "",
    "a",
    "ab",
    "bba",
    "baac",
    "bc",
    "ca",
    "cba",
    "abc",
    "abbcc",
    "aabcc",
    "abbcccc",
    "aabbbcccccc",
    "abbbbbccccc",
    "aaabbbccccccccc",
    "aaabbcccccc",
]

create_and_test_turing_machine(tm_M3,tm_M3_test,verbose='verbose')




*** TEST RUNS ***
q1:	[_]
REJECT input 

q1:	[a]
REJECT input a

q1:	[a]b
REJECT input ab

q1:	[b]ba
REJECT input bba

q1:	[b]aac
REJECT input baac

q1:	[b]c
REJECT input bc

q1:	[c]a
REJECT input ca

q1:	[c]ba
REJECT input cba

q1:	[a]bc
ACCEPT input abc

q1:	[a]bbcc
ACCEPT input abbcc

q1:	[a]abcc
ACCEPT input aabcc

q1:	[a]bbcccc
REJECT input abbcccc

q1:	[a]abbbcccccc
ACCEPT input aabbbcccccc

q1:	[a]bbbbbccccc
ACCEPT input abbbbbccccc

q1:	[a]aabbbccccccccc
ACCEPT input aaabbbccccccccc

q1:	[a]aabbcccccc
ACCEPT input aaabbcccccc



*** RUN COMPLETE ***




## Exercise B

In [4]:

# Turing Machine M4, Sipser 3ird ed page 175, Sipser 2nd ed page 147

tm_M4 = """# The language of distinct elements
# &x1&x2&...&xk where each xi in {0,1}*, and xi != xj for each i != j

# student assignment 
# note: the book says to place a mark on top of a &. let the tape symbol for a
# "marked" & be a %.

start: q1
accept: A
reject: R

state: q1
    _ : r A
    & % r q2
    
state: q2
    0 : r q2
    1 : r q2
    & % r q3
    _ : r A

state: q3
    Z : r q3
    O : r q3
    0 Z l q4
    1 O l q5
    & : l q14
    _ : l q14
    
state: q4
    0 : l q4
    1 : l q4
    & : l q4
    % : l q16
    Z : l q4
    O : l q4
    
state: q16
    0 : l q16
    1 : l q16
    & : l q16
    % : r q6
    Z : r q6
    O : r q6

state: q5
    0 : l q5
    1 : l q5
    & : l q5
    % : l q17
    Z : l q5
    O : l q5
    
state: q17
    0 : l q17
    1 : l q17
    & : l q17
    % : r q7
    Z : r q7
    O : r q7
    
state: q6
    0 Z r q8
    1 : l q9
    & : l q9
    % : l q9

state: q7
    0 : l q9
    1 O r q8
    & : l q9
    % : l q9

state: q8
    0 : r q8
    1 : r q8
    & : r q8
    % : r q3
    
state: q9
    Z 0 l q9
    O 1 l q9
    % : r q10

state: q10
    0 : r q10
    1 : r q10
    & : r q10
    % & r q11
    
state: q11
    Z 0 r q11
    O 1 r q11
    0 : r q11
    1 : r q11
    & % r q3
    _ : l q12
    
state: q12
    0 : l q12
    1 : l q12
    & : l q12
    % & r q13

state: q13
    0 : r q13
    1 : r q13
    & % r q2

state: q14
    0 : l q14
    1 : l q14
    & : l q14
    % : l q15
    Z : l q14
    O : l q14
    
state: q15
    0 : l q15
    1 : l q15
    & : l q15
    % : r q18
    Z : r q18
    O : r q18
    
state: q18
    0 : l q9
    1 : l q9
"""
tm_M4_test = [
    "0",
    "1",
    "",
    "&",
    "&0",
    "&1",
    "&00",
    "&01",
    "&10",
    "&11",
    "&&0",
    "&&1",
    "&0&",
    "&1&"
    
 
]

create_and_test_turing_machine(tm_M4,tm_M4_test,verbose='none')




*** TEST RUNS ***
q1:	[0]
REJECT input 0

q1:	[1]
REJECT input 1

q1:	[_]
ACCEPT input 

q1:	[&]
ACCEPT input &

q1:	[&]0
ACCEPT input &0

q1:	[&]1
ACCEPT input &1

q1:	[&]00
ACCEPT input &00

q1:	[&]01
ACCEPT input &01

q1:	[&]10
ACCEPT input &10

q1:	[&]11
ACCEPT input &11

q1:	[&]&0
ACCEPT input &&0

q1:	[&]&1
ACCEPT input &&1

q1:	[&]0&
ACCEPT input &0&

q1:	[&]1&
ACCEPT input &1&



*** RUN COMPLETE ***




## Exercise C

Write and test TM's for the following languages over the alphabet { 0, 1 },

1. all strings w that contain equal numbers of 0s and 1s
1. all strings w that contain twice as many 0s as 1s
1. all strings w that do not contain twice as many 0s as 1s.


In [5]:
tm_3_8_a = """# exercise 3.8(a) in sipser

start: q1
accept: A
reject: R

state: q1
    0 % r q2
    1 % r q3
    _ : r A
    x : r q1
    
state: q2
    1 x l q4
    0 : r q2
    x : r q2
    
state: q3
    1 : r q3
    0 x l q4
    x : r q3
    
state: q4
    0 : l q4
    1 : l q4
    x : l q4
    % : r q1
"""


tm_3_8_b = """# exercise 3.8(b) in sipser

start: q1
accept: A
reject: R

state: q1
    0 % r q2
    1 % r q3
    _ : r A
    x : r q1

state: q2
    0 x r q4
    1 x r q5
    x : r q2

state: q3
    0 x r q6
    1 : r q3
    x : r q3

state: q4
    1 x l q7
    0 : r q4
    x : r q4
    
state: q5
    0 x l q7
    1 : r q5
    x : r q5
    
state: q6
    0 x l q7
    1 : r q6
    x : r q6
    
state: q7
    0 : l q7
    1 : l q7
    x : l q7
    % : r q1
"""
tm_3_8_b_test = [
    "110000000011",
]

create_and_test_turing_machine(tm_3_8_b,tm_3_8_b_test,verbose='explain')

tm_3_8_c = """# exercise 3.8(c) in sipser

start: q1
accept: A
reject: R

state: q1
    0 % r q2
    1 % r q3
    x : r q1
    
state: q2
    0 x r q4
    1 x r q5
    x : r q2
    _ : r A

state: q3
    0 x r q6
    1 : r q3
    x : r q3
    _ : r A

state: q4
    1 x l q7
    0 : r q4
    x : r q4
    _ : r A
    
state: q5
    0 x l q7
    1 : r q5
    x : r q5
    _ : r A
    
state: q6
    0 x l q7
    1 : r q6
    x : r q6
    _ : r A
    
state: q7
    0 : l q7
    1 : l q7
    x : l q7
    % : r q1
"""

tm_3_8_c_test = [
    "000111",
]

create_and_test_turing_machine(tm_3_8_c,tm_3_8_c_test,verbose='explain')


*** TEST RUNS ***
q1:	[1]10000000011
q3:	%[1]0000000011
q3:	%1[0]000000011
q6:	%1x[0]00000011
q7:	%1[x]x00000011
q7:	%[1]xx00000011
q7:	[%]1xx00000011
q1:	%[1]xx00000011
q3:	%%[x]x00000011
q3:	%%x[x]00000011
q3:	%%xx[0]0000011
q6:	%%xxx[0]000011
q7:	%%xx[x]x000011
q7:	%%x[x]xx000011
q7:	%%[x]xxx000011
q7:	%[%]xxxx000011
q1:	%%[x]xxx000011
q1:	%%x[x]xx000011
q1:	%%xx[x]x000011
q1:	%%xxx[x]000011
q1:	%%xxxx[0]00011
q2:	%%xxxx%[0]0011
q4:	%%xxxx%x[0]011
q4:	%%xxxx%x0[0]11
q4:	%%xxxx%x00[1]1
q7:	%%xxxx%x0[0]x1
q7:	%%xxxx%x[0]0x1
q7:	%%xxxx%[x]00x1
q7:	%%xxxx[%]x00x1
q1:	%%xxxx%[x]00x1
q1:	%%xxxx%x[0]0x1
q2:	%%xxxx%x%[0]x1
q4:	%%xxxx%x%x[x]1
q4:	%%xxxx%x%xx[1]
q7:	%%xxxx%x%x[x]x
q7:	%%xxxx%x%[x]xx
q7:	%%xxxx%x[%]xxx
q1:	%%xxxx%x%[x]xx
q1:	%%xxxx%x%x[x]x
q1:	%%xxxx%x%xx[x]
q1:	%%xxxx%x%xxx[_]
A:	%%xxxx%x%xxx_[_]
ACCEPT input 110000000011



*** RUN COMPLETE ***



*** TEST RUNS ***
q1:	[0]00111
q2:	%[0]0111
q4:	%x[0]111
q4:	%x0[1]11
q7:	%x[0]x11
q7:	%[x]0x11
q7:	[%]x0x11
q1:	%[x]0x11
q1:	%x